In [1]:
import labelbox as lb
import labelbox.types as lb_types
import uuid
import base64
import requests

# Setup client
with open("labelbox_api_key.txt","r") as f:
    API_KEY = f.read().strip()
client = lb.Client(api_key=API_KEY)

# Get ontology
print("===ONTOLOGY DETAILS===")
ontology = client.get_ontology("clqo6bd8v0jc407ybc1r9ehlb")
print("Name: ", ontology.name)
tools = ontology.tools()

# for tool in tools:
#   print(tool)

# Get project
print("\n===PROJECT DETAILS===")
PROJECT_ID = 'clqoh3ylw1o8s070hd6ch5z7o' # WHOI RSI USVI Fish
# PROJECT_ID = 'clqo7auln0mpo07wphorp0t2e' # Test WHOI RSI USVI Fish
project = client.get_project(PROJECT_ID)
print("Name: ", project.name)

# Get dataset
DATASET_ID = "clqh7v7qi001r07886j6aws7i"
dataset = client.get_dataset(DATASET_ID)
print("\n===DATASET DETAILS===")
print("Name: ", dataset.name)

===ONTOLOGY DETAILS===
Name:  WHOI-RSI-USVI-Fish

===PROJECT DETAILS===
Name:  WHOI-RSI-USVI-Fish-detect-and-track

===DATASET DETAILS===
Name:  imerit-26102023-3fps-clips


In [7]:
## General fish detector first
# Extract species and associate ids:
# for option in tools[0].classifications[0].options:
#     print(option.label)


In [10]:
## Recommended to download JSON from Labelbox using the Browser Interface
# TODO: Verify whether or not this includes interpolated (non-keyframed) data

# Extracts fish labels from labelbox json file and converts them into YOLO format
# Assumes the global_key from labelbox matches the directory structure of the images
# Fish class is assumed as 0

import json
import jsonlines
import os
from pathlib import Path
from bbox_utils import *
from tqdm import tqdm

json_path = "/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-labels-04042024.json"
image_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/images")
label_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/labels")

# with open(json_path, "r") as f:
with jsonlines.open(json_path) as f:

    # Iterate through each video
    for i, datarow in tqdm(enumerate(f)):
        global_key = datarow["data_row"]["global_key"]

        img_sz = (datarow["media_attributes"]["width"], datarow["media_attributes"]["height"])
        
        # Video path
        vid_path = Path(global_key)
        rel_vid_path = vid_path.parent / "_".join(vid_path.stem.split("_")[:-1])

        # Grab frame labels
        try:
            frames_json = datarow["projects"][PROJECT_ID]["labels"][0]["annotations"]["frames"]
        except:
            print("Skipping ", global_key, " has no labels")
            continue

        # Iterate through frames
        for frame_id, frame_data in frames_json.items():
            img_name = "frame_%03d"%(int(frame_id)-1)

            img_path = image_root / rel_vid_path / (img_name + ".png")
            output_path = label_output_root / rel_vid_path / (img_name + ".txt")

            # Verify this image exists
            assert(img_path.exists(), f"Image not found {img_path}")

            os.makedirs(output_path.parent, exist_ok=True)
            
            for object_id, object_data in frame_data["objects"].items():
                lbl_bbox = object_data["bounding_box"]

                # print(object_data["name"])
                
                # if len(object_data["classifications"]) > 0:
                #     print(object_data["classifications"])

                yolo_bbox = list(labelbox2yolo_bbox(lbl_bbox, img_sz))
                yolo_bbox.insert(0,0) # Fish class for now
                with open(output_path, "a") as f:
                    f.write(" ".join(map(str, yolo_bbox)))
                    f.write("\n")

print("done")

<>:47: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:47: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/tmp/ipykernel_2009170/2880659347.py:47: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(img_path.exists(), f"Image not found {img_path}")
12it [01:09,  5.61s/it]

Skipping  Summer2016/JoelsShoal30mTransects061016/P6100005_0m_20s_aws150.mp4  has no labels


163it [06:04,  2.24s/it]

done


In [11]:
## Recommended to download JSON from Labelbox using the Browser Interface
# TODO: Verify whether or not this includes interpolated (non-keyframed) data

# Extracts fish labels from labelbox json file and converts them into YOLO format
# Assumes the global_key from labelbox matches the directory structure of the images
# Fish class is assumed as 0

import json
import jsonlines
import os
from pathlib import Path
from bbox_utils import *
from tqdm import tqdm

json_path = "/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-labels-04042024.json"
image_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/images")
label_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/labels")

# with open(json_path, "r") as f:
with jsonlines.open(json_path) as f:

    # Iterate through each video
    for i, datarow in tqdm(enumerate(f)):
        global_key = datarow["data_row"]["global_key"]

        img_sz = (datarow["media_attributes"]["width"], datarow["media_attributes"]["height"])
        
        # Video path
        vid_path = Path(global_key)
        rel_vid_path = vid_path.parent / "_".join(vid_path.stem.split("_")[:-1])

        # Grab frame labels
        try:
            frames_json = datarow["projects"][PROJECT_ID]["labels"][0]["annotations"]["frames"]
        except:
            print("Skipping ", global_key, " has no labels")
            continue

        # Iterate through frames
        # TODO: brittle, instead iterate through known images
        for frame_id in range(90):
            img_name = "frame_%03d"%(int(frame_id)-1)
            img_path = image_root / rel_vid_path / (img_name + ".png")
            output_path = label_output_root / rel_vid_path / (img_name + ".txt")  
            
            # Verify this image exists
            assert(img_path.exists(), f"Image not found {img_path}")
            
            os.makedirs(output_path.parent, exist_ok=True)
            
            # Make label file, overwrite if already there
            open(output_path, "w")
            
            if str(frame_id) in frames_json:
                frame_data = frames_json[str(frame_id)]
            else:
                continue

            for object_id, object_data in frame_data["objects"].items():
                lbl_bbox = object_data["bounding_box"]

                # print(object_data["name"])
                
                # if len(object_data["classifications"]) > 0:
                #     print(object_data["classifications"])

                yolo_bbox = list(labelbox2yolo_bbox(lbl_bbox, img_sz))
                yolo_bbox.insert(0,0) # Fish class for now
                with open(output_path, "a") as f:
                    f.write(" ".join(map(str, yolo_bbox)))
                    f.write("\n")

print("done")

<>:47: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:47: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/tmp/ipykernel_2009170/1460734442.py:47: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(img_path.exists(), f"Image not found {img_path}")
12it [01:03,  4.87s/it]

Skipping  Summer2016/JoelsShoal30mTransects061016/P6100005_0m_20s_aws150.mp4  has no labels


163it [06:01,  2.22s/it]

done


In [13]:
# Create yolov5 dataset configuration yaml
import yaml

project_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/")
train_split_filename = "train_split.txt"
val_split_filename = "val_split.txt"
test_split_filename = "test_split.txt"

yolo_dataset = {
    "path": str(project_root),
    "train": f"./{train_split_filename}",
    "val": f"./{val_split_filename}",
    "test": f"./{test_split_filename}",
    "names": {0: "fish"},
}
yaml_data = yaml.dump(yolo_dataset)
with open(project_root / "yolo_dataset.yaml", "w") as f:
    f.write(yaml_data)

In [22]:
# Create train, val, and test splits
open(project_root / train_split_filename, "w")
open(project_root / val_split_filename, "w")
open(project_root / test_split_filename, "w")

import glob

# Test split contains only years 2016 and 2017, these dates are inferred from the global_key
img_paths = glob.iglob(str(project_root / "**/*.png"), recursive=True)

for img_path in tqdm(img_paths):
    # Get relative path starting at project root
    project_root_parts = len(project_root.parts)
    
    project_img_path = Path(*Path(img_path).parts[project_root_parts:])
    
    vid_path = project_img_path.parent
    
    with open(project_root / "video_list.txt", "a") as f:
        f.write(str(vid_path) + "\n")
        
    # Test split
    if "2016" in str(project_img_path) or "2017" in str(project_img_path):
        with open(project_root / test_split_filename, "a") as f:
            f.write("./" + str(project_img_path) + "\n")
            
    # Val split
    elif "2018" in str(project_img_path):
        with open(project_root / val_split_filename, "a") as f:
            f.write("./" + str(project_img_path) + "\n")
    
    # Train split
    else:
        with open(project_root / train_split_filename, "a") as f:
            f.write("./" + str(project_img_path) + "\n")
        
print("done")


14670it [01:22, 177.71it/s]

done
